In [1]:
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers
import keras
import pandas
import utils

Using TensorFlow backend.


In [18]:
train_folder = 'data/train'
test_folder = 'data/test'
validation_folder = 'data/validation'
labels_csv = 'train_labels.csv'
label_names = ['0', '1']
IMG_WIDTH = 193
IMG_HEIGHT = 144
batch_size = 4

In [19]:
utils.create_labelled_folders(train_folder, labels_csv, label_names)        
utils.create_validation_subfolders(train_folder, validation_folder, label_names, 0.2)        

Folders are already labelled


In [20]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    rescale=1./255.,
    horizontal_flip=True,
)

validation_datagen = ImageDataGenerator(
    rescale=1./255.
)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    batch_size=batch_size,
    target_size = (IMG_WIDTH, IMG_HEIGHT)
)

validation_generator = validation_datagen.flow_from_directory(
    validation_folder,
    batch_size=batch_size,
    target_size = (IMG_WIDTH, IMG_HEIGHT)
)

In [24]:
base_model = keras.applications.vgg16.VGG16(weights='imagenet', 
                                            include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

In [25]:
x = base_model.output
x = (Flatten(input_shape=base_model.output_shape[1:], name="finetune"))(x)
x = (Dense(256, activation='relu'))(x)
x = (Dropout(0.5))(x)
x = (Dense(2, activation='sigmoid'))(x)

model = Model(inputs=base_model.inputs, outputs=x)

for layer in model.layers:
    if layer.name == "finetune":
        break
    
    layer.trainable = False

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [27]:
num_training_samples = train_generator.n
num_validation_samples = validation_generator.n

model.fit_generator(
    train_generator,
    steps_per_epoch=num_training_samples//batch_size,
    epochs=1,
    validation_data=validation_generator,
    validation_steps=num_validation_samples//batch_size)

Epoch 1/1
459/459 [==============================] - 212s - loss: 0.6134 - acc: 0.6669 - val_loss: 0.5330 - val_acc: 0.7840


In [28]:
test_datagen = ImageDataGenerator(
    rescale=1./255.
)

test_generator = test_datagen.flow_from_directory(
    test_folder,
    batch_size=batch_size,
    target_size = (IMG_WIDTH, IMG_HEIGHT)
)

Found 1531 images belonging to 1 classes.


In [29]:
num_test_samples = test_generator.n
test_predictions = model.predict_generator(test_generator, num_test_samples//batch_size)

In [31]:
test_predictions[:,:20]

array([[ 0.5857569 ,  0.37156978],
       [ 0.50688696,  0.50835228],
       [ 0.63234729,  0.28330767],
       ..., 
       [ 0.24424064,  0.74946123],
       [ 0.67320842,  0.33773559],
       [ 0.51882637,  0.382276  ]], dtype=float32)